In [1]:
import pandas as pd
import re
import seaborn as sb
import matplotlib.pyplot as plt
import datetime
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
df_movies = pd.read_csv("the-movies-dataset/movies_metadata.csv")
print("Length at import: " + str(len(df_movies)))
#df_movies.head(3)
# filter movies on Status == 'Released' and drop status column
df_movies = df_movies[df_movies["status"] == 'Released']
df_movies.drop(columns=["status"])
print("Length only released: " + str(len(df_movies)))
#df_movies = df_movies[0:1000]
print("Length: " + str(len(df_movies)))
df_movies_raw = df_movies.copy()

Length at import: 45466
Length only released: 45014
Length: 45014


C:\Users\d060445\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
genres_stat = {}
for index, row in df_movies.iterrows():
    genres = json.loads(re.sub("'", '"',row.genres))
    for genre in genres: # turn string to json and loop over genres
        genres_stat.update({genre.get("name") : 0})

#Get the movie count per genre
movies = {}
for index, row in df_movies.iterrows():
    tmp_genres = []  
    genres = json.loads(re.sub("'", '"',row.genres))
    for genre in genres:
        tmp_genres.append(genre.get("name"))
        new_value = genres_stat.get(genre.get("name")) + 1
        genres_stat.update({genre.get("name") : new_value})
    movies.update({row.id:{'title': row.title, 'genres': tmp_genres}})
print(len(movies))

for key, value in sorted(genres_stat.items(), key=lambda item: item[1]):
    print("%s: %s" % (key, value))

44985
TV Movie: 759
Western: 1038
War: 1318
History: 1391
Foreign: 1588
Music: 1588
Animation: 1916
Fantasy: 2294
Mystery: 2456
Family: 2743
Science Fiction: 3018
Adventure: 3474
Documentary: 3874
Crime: 4286
Horror: 4636
Action: 6546
Romance: 6668
Thriller: 7571
Comedy: 13079
Drama: 20078


In [4]:
# one hot encoding of genres (and other list columns)
# one hot encoding of genres: (takes ~3-4 minutes)
# 1. get genres
#genre_set = {"Action","Drama","Comedy","Thriller","Animation"}

genre_set = {""}
for index, row in df_movies.iterrows():
    genres = json.loads(re.sub("'", '"',row.genres))
    for genre in genres: # turn string to json and loop over genres
        genre_set.add(genre.get("name"))       
genre_set.remove("")

for g in genre_set:
    df_movies[g] = 0

# 2. add genres as columns to dataframe, default value = 0
df_movie_genres = pd.DataFrame(columns=genre_set)
for index, row in df_movies.iterrows():
    temp = pd.Series(index=genre_set)
    tmp_genres = []  
    genres = json.loads(re.sub("'", '"',row.genres))
    for genre in genres:
        if genre.get("name") in list(genre_set):
            tmp_genres.append(genre.get("name"))
    for genre in tmp_genres: # turn string to json and loop over genres
        #temp[genre] = (1/len(tmp_genres))
        temp[genre] = 1
    df_movie_genres = df_movie_genres.append(temp, ignore_index=True)
#print("One hot encoding finished")

# 3. add one hot encoded genres to df_movies        
df_movies[list(genre_set)] = df_movie_genres

# 4. turn NaN values to zeros
df_movies = df_movies.fillna(0)

df_movies.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,Animation,Adventure,Music,Mystery,History,Romance,Family,Crime,Western,Fantasy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
#df_movies = df_movies.drop(columns=['genres','imdb_id','original_title','overview','popularity','poster_path','release_date','revenue','status','tagline','title','video','vote_count','vote_average'])
df_movies = df_movies.drop(columns=['genres','original_title','overview','popularity','poster_path','release_date','revenue','status','tagline','title','video','vote_average'])
df_movies.head(3)

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,production_companies,production_countries,runtime,...,Animation,Adventure,Music,Mystery,History,Romance,Family,Crime,Western,Fantasy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",81.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,False,0,65000000,0,8844,tt0113497,en,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",104.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,0,15602,tt0113228,en,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",101.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [6]:
# durchschnitt von predictions berechnen
df_links = pd.read_csv("the-movies-dataset/links.csv")
print("Length at import (Links): " + str(len(df_links)))

df_ids = pd.DataFrame(columns=['ratingID'])
df_ids.ratingID = df_ids.ratingID.astype('int64')
pattern = "^tt(\d*)"
for index, row in df_movies.iterrows():
    temp = pd.Series(['ratingID'])
    newValue = str(row.imdb_id).replace("t","",-1)
    temp['ratingID'] = int(newValue)
    df_ids = df_ids.append(temp, ignore_index=True)
df_movies['ratingID'] = df_ids['ratingID']
print("Länge: ", len(df_ids['ratingID']))

df_tmp = pd.merge(left=df_movies,right=df_links, left_on='ratingID', right_on='imdbId', how='inner')
#print(df_tmp.head(5))
print("Length after Merge (Movies + Links): " + str(len(df_tmp)))

#######
df_ratings = pd.read_csv("the-movies-dataset/ratings.csv")
#df_ratings = df_ratings[0:100000]
print("Length at import (Ratings): " + str(len(df_ratings)))
df_ratings_grouped = df_ratings.groupby('movieId')[['rating']].mean()
#print(df_ratings_grouped.head(5))
df_joined = pd.merge(left=df_tmp,right=df_ratings_grouped, left_on='movieId', right_on='movieId', how='inner')
print("Length after merge (Movies + Links + Ratings): " + str(len(df_joined)))

Length at import (Links): 45843
Länge:  45014
Length after Merge (Movies + Links): 44497
Length at import (Ratings): 26024289
Length after merge (Movies + Links + Ratings): 43792


In [7]:
## Export: ID + Genres
df_export = df_joined.copy()
features_to_remove = ['id','imdb_id','movieId','tmdbId','ratingID','production_countries','production_companies','adult','belongs_to_collection','budget','homepage','runtime','original_language','spoken_languages','vote_count']
for i in features_to_remove:
    if i in df_export.columns:
        df_export = df_export.drop(columns=i)
df_export.to_csv("clusterPreprocessing.csv", index=False)

In [8]:
# cast und crew integrieren
df_credits = pd.read_csv("the-movies-dataset/credits.csv")
df_credits.head(5)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [9]:
pattern = '\"Director\", \"name\": \"([a-zA-Z ]*)\", \"'
directors = set()
df_credits['director'] = ""
for index, row in df_credits.iterrows():
    text = row.crew
    text = text.replace('\\','',-1)
    text = text.replace("'",'"',-1)
    a = re.search(pattern, text)
    if a != None:
        directors.add(a.group(1))
        df_credits.set_value(index, 'director', a.group(1))
print(len(directors))
#print(directors)
df_credits.head(10)

14945


,cast,crew,id,director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,John Lasseter
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Joe Johnston
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Howard Deutch
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Forest Whitaker
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,Charles Shyer
5,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",949,Michael Mann
6,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",11860,Sydney Pollack
7,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",45325,Peter Hewitt
8,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",9091,Peter Hyams
9,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",710,Martin Campbell


In [10]:
import json
pattern = '\"name\": \"(.*)\", '
df_credits['actors'] = ""
for index, row in df_credits.iterrows():
    actors = []
    text = row.cast
    text = text.replace('\\','',-1)
    text = text.replace("'",'"',-1)
    liste = text.split("cast_id")
    for i in liste:
        tmp_name = re.search(pattern, i)
        if tmp_name != None:
            actors.append(tmp_name.group(1))
    df_credits.set_value(index, 'actors', ','.join(map(str, actors)))
df_credits.head(10)

,cast,crew,id,director,actors
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,John Lasseter,"Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Joe Johnston,"Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Howard Deutch,"Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ..."
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Forest Whitaker,"Whitney Houston,Angela Bassett,Loretta Devine,..."
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,Charles Shyer,"Steve Martin,Diane Keaton,Martin Short,Kimberl..."
5,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",949,Michael Mann,"Al Pacino,Robert De Niro,Val Kilmer,Jon Voight..."
6,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",11860,Sydney Pollack,"Harrison Ford,Julia Ormond,Greg Kinnear,Angie ..."
7,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",45325,Peter Hewitt,"Jonathan Taylor Thomas,Brad Renfro,Rachael Lei..."
8,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",9091,Peter Hyams,"Jean-Claude Van Damme,Powers Boothe,Dorian Har..."
9,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",710,Martin Campbell,"Pierce Brosnan,Sean Bean,Izabella Scorupco,Fam..."


In [11]:
df_credits = df_credits.drop(columns=['cast','crew'])
df_joined = pd.merge(left=df_joined,right=df_credits, left_on='id', right_on='id', how='inner')
print("Length after merge (Movies + Ratings + Credits): " + str(len(df_joined)))
df_joined.head(5)

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
# belongs to, homepage --> binär
df_joined['part_of_collection'] = ""
df_joined['+18'] = ""
df_joined['hasHomepage'] = ""
for index, row in df_joined.iterrows():
    if row.belongs_to_collection == 0:
        df_joined.set_value(index, 'part_of_collection', 0)
    else:
        df_joined.set_value(index, 'part_of_collection', 1)
    
    if row.adult == "True":
        df_joined.set_value(index, '18+', 1)
    else:
        df_joined.set_value(index, '18+', 0)   
    
    pattern = 'www.'
    tmp = re.search(pattern, str(row.homepage))
    if tmp != None:
        df_joined.set_value(index, 'hasHomepage', 1)
    else:
        df_joined.set_value(index, 'hasHomepage', 0)
df_joined = df_joined.drop(columns=['belongs_to_collection','adult','homepage'])
df_joined.head(5)

In [ ]:
import json
pattern1 = "{'name': '([a-zA-Z ]*)'"
pattern2 = "'name': '([a-zA-Z ]*)'"

df_joined['productionCompanies'] = ""
df_joined['productionCountries'] = ""
df_joined['spokenLanguages'] = ""
for index, row in df_joined.iterrows():
    companies = []
    countries = []
    lang = []
    
    text1 = row.production_companies
    liste1 = text1.split("}")
    for i in liste1:
        tmp_name = re.search(pattern1, i)
        if tmp_name != None:
            companies.append(tmp_name.group(1))
    df_joined.set_value(index, 'productionCompanies', ','.join(map(str, companies)))
    
    text2 = row.production_countries
    liste2 = text2.split("}")
    for i in liste2:
        tmp_name = re.search(pattern2, i)
        if tmp_name != None:
            countries.append(tmp_name.group(1))
    df_joined.set_value(index, 'productionCountries', ','.join(map(str, countries)))
    
    text3 = row.spoken_languages
    liste3 = text3.split("}")
    for i in liste3:
        tmp_name = re.search(pattern2, i)
        if tmp_name != None:
            lang.append(tmp_name.group(1))
    df_joined.set_value(index, 'spokenLanguages', ','.join(map(str, lang)))
    
    
df_joined.head(5)

In [ ]:
df_joined = df_joined.drop(columns=['production_companies','production_countries','spoken_languages'])
df_joined.head(5)

### production companies, production countries, spoken_languages,original language --> hot encoding?
### what about continuous attributes? --> budget

In [ ]:
# how many different original languages? *
print(str(len(df_joined.original_language.unique())) + " original languages")

# how many different spoken languages?
print(str(len(df_joined.spokenLanguages.unique())) + " spoken languages")

# how many different production companies? *
print(str(len(df_joined.productionCompanies.unique())) + " production companies")

# how many different production countries?
print(str(len(df_joined.productionCountries.unique())) + " production countries")

# max budget?
print("Max Budget: " + str(df_joined.budget.max()))

# min budget?
print("Min Budget: " + str(df_joined.budget.min()))

### Hot Encoding for original languages (orig)

In [ ]:
# lang_set = {""}
# for index, row in df_joined.iterrows():
#     lang_set.add("orig_" + str(row.original_language))      
# lang_set.remove("")

# for g in lang_set:
#     df_joined[g] = 0

# df_movie_lang = pd.DataFrame(columns=lang_set)
# for index, row in df_joined.iterrows():
#     temp = pd.Series(index=lang_set)
#     tmp_lang = "orig_" + str(row.original_language)
#     temp[tmp_lang] = 1
#     df_movie_lang = df_movie_lang.append(temp, ignore_index=True)

# # 3. add one hot encoded genres to df_movies
# df_joined[list(lang_set)] = df_movie_lang

# # 4. turn NaN values to zeros
# df_joined = df_joined.fillna(0)

# df_joined = df_joined.drop(columns=['original_language'])

# df_joined.head(3)

### Hot Encoding for Production Countries (pcountry)

In [ ]:
# df_movies_raw, am Ende --> df_joined
pc_set = {""}
for index, row in df_joined.iterrows():
    pcs = str(row.productionCountries).split(",")
    for pc in pcs: # turn string to json and loop over genres
        pc_set.add("pcountry_" + str(pc))       
pc_set.remove("")

for g in pc_set:
    df_joined[g] = 0

# 2. add genres as columns to dataframe, default value = 0
df_movie_pc = pd.DataFrame(columns=pc_set)
for index, row in df_joined.iterrows():
    temp = pd.Series(index=pc_set)
    tmp_pcs = []
    pcs = str(row.productionCountries).split(",")
    for pc in pcs:
        tmp_pcs.append("pcountry_" + str(pc))
    for pc in tmp_pcs:
        #temp[pc] = (1/len(tmp_pcs))
        temp[pc] = 1
    df_movie_pc = df_movie_pc.append(temp, ignore_index=True)

# 3. add one hot encoded genres to df_movies        
df_joined[list(pc_set)] = df_movie_pc

# 4. turn NaN values to zeros
df_joined = df_joined.fillna(0)

df_joined = df_joined.drop(columns=['productionCountries'])

df_joined.head(3)

### Hot Encoding for Production Companies (pcomp)

In [ ]:
pland_set = {""}
for index, row in df_joined.iterrows():
    pcs = str(row.productionCompanies).split(",")
    for pc in pcs: # turn string to json and loop over genres
        pland_set.add("pcomp_" + str(pc))       
pland_set.remove("")

for g in pland_set:
    df_joined[g] = 0

# 2. add genres as columns to dataframe, default value = 0
df_movie_pc = pd.DataFrame(columns=pland_set)
for index, row in df_joined.iterrows():
    temp = pd.Series(index=pland_set)
    tmp_pcs = []
    pcs = str(row.productionCompanies).split(",")
    for pc in pcs:
        tmp_pcs.append("pcomp_" + str(pc))
    for pc in tmp_pcs:
        #temp[pc] = (1/len(tmp_pcs))
        temp[pc] = 1
    df_movie_pc = df_movie_pc.append(temp, ignore_index=True)

# 3. add one hot encoded genres to df_movies        
df_joined[list(pland_set)] = df_movie_pc

# 4. turn NaN values to zeros
df_joined = df_joined.fillna(0)

df_joined = df_joined.drop(columns=['productionCompanies'])

df_joined.head(3)

### Hot Encoding for spoken Languages (slang)

In [ ]:
slang_set = {""}
for index, row in df_joined.iterrows():
    pcs = str(row.spokenLanguages).split(",")
    for pc in pcs: # turn string to json and loop over genres
        slang_set.add("slang_" + str(pc))       
slang_set.remove("")

for g in slang_set:
    df_joined[g] = 0

# 2. add genres as columns to dataframe, default value = 0
df_movie_pc = pd.DataFrame(columns=slang_set)
for index, row in df_joined.iterrows():
    temp = pd.Series(index=slang_set)
    tmp_pcs = []
    pcs = str(row.spokenLanguages).split(",")
    for pc in pcs:
        tmp_pcs.append("slang_" + str(pc))
    for pc in tmp_pcs:
        #temp[pc] = (1/len(tmp_pcs))
        temp[pc] = 1
    df_movie_pc = df_movie_pc.append(temp, ignore_index=True)

# 3. add one hot encoded genres to df_movies        
df_joined[list(slang_set)] = df_movie_pc

# 4. turn NaN values to zeros
df_joined = df_joined.fillna(0)

df_joined = df_joined.drop(columns=['spokenLanguages'])

df_joined.head(3)

### director, actors?
##### auch Anzahl der Ratings kann eine Rolle spielen

In [13]:
# production companies, production countries, spoken_languages --> hot encoding?
# first get list of all actoors
df_credits1 = df_credits
dict_actors = {}
occurences_by_actor= []
for index, row in df_credits1.iterrows():
    liste_actors = row.actors.split(",")
    for i in liste_actors:
        if i in dict_actors:
            dict_actors[i] = dict_actors[i] + 1
        else:
            dict_actors[i] = 1

# now that we got a list let's check how many actors there are and in how many movies they played
print("Overall NUmber of Actors:")
print(len(dict_actors))
print("Number of different occurances:")
print(list(set(dict_actors.values())))
            
# what about continuous attributes? --> budget
# original language: en = 1, else=0?

del dict_actors['']
del dict_actors[' Jr.']
df_actors_plays = pd.DataFrame.from_dict(list(dict_actors.items()))

df_actors_plays.columns

#print(df_actors_plays.sort_values(by=1, ascending=False))
print(df_actors_plays.count())

df_actors_plays1 = df_actors_plays[df_actors_plays[1] > 70]
print(df_actors_plays1.count())



#print(df_actors_plays1.sort_values(by=1, ascending=False))

#df_actors_plays1.sort_values(by=1).plot(kind='line',x=0, y=1, figsize=(10, 50))

Overall NUmber of Actors:
202735
Number of different occurances:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 96, 97, 98, 101, 102, 104, 107, 108, 109, 110, 123, 125, 148, 241, 305, 2420]
0    202733
1    202733
dtype: int64
0    94
1    94
dtype: int64


In [ ]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
director_target = label_encoder.fit_transform(df_joined['director'])
df_joined = df_joined.drop(columns=['director'])
df_joined['director'] = director_target

In [ ]:
print("Length before export: " + str(len(df_joined)))
# save to file
df_joined.to_csv("regressionPreprocessing.csv", index=False)